# Resolució de problemes de cerca

En aquest `notebook` definirem un procés per resoldre problemes de cerca (on vullgam coneixer el camí per arribar a un estat objectiu). Utilitzarem el problema de les torres de Hanoi per a ilustrar el procés.

## Torres de Hanoi

### Definició de l'estat

Per a poder resoldre un problema de cerca, primer hem de definir l'estat del problema. En aquest cas, l'estat del problema és la posició dels discos en les torres. Per això, definirem una classe amb aquesta informació.

Utilitzarem una dataclass de Python per a definir l'estat. Una dataclass és una classe que té un constructor que inicialitza els atributs de la classe. Això ens permetrà crear instàncies de la classe amb una sintaxi més senzilla. 

In [210]:
from dataclasses import dataclass


@dataclass
class EstatHanoi(object):
    """
    Estat del problema de les torres de Hanoi: guardarem un array amb els discos de cada torre i un punter a l'estat pare.
    Definirem també el mètode __str__ per a poder imprimir l'estat, el hash per a poder utilitzar-lo en un set, i els mètodes __eq__ i __lt__ per a poder comparar dos estats.
    """
    torres: list
    pare: 'EstatHanoi' = None

    def __repr__(self):
        """
        Retorna una cadena de caràcters amb la representació de l'estat.
        """
        out = ""
        for torre in self.torres:
            out += str(torre) + "\n"
        out += "-----\n"
        return out

    def __hash__(self):
        """
        Retorna un hash (nombre enter cqlculat) que representa l'estat.
        """
        return hash(str(self.torres))

    def __eq__(self, altre):
        """
        Retorna True si l'estat és igual a l'estat `altre`. No inclou el pare.
        """
        return self.torres == altre.torres

    def __lt__(self, altre):
        """
        Retorna True si l'estat és menor que l'estat `altre`. No inclou el pare.
        """
        return self.torres < altre.torres



### Definició del problema

Per simplificar el procés utilitzarem una classe que ens permeti definir problemes. El tindre una classe amb aquesta funcionalitat ens permetrà reutilitzar el codi per a diferents problemes.

In [211]:
class Problema(object):
    """Aquesta és la classe abstracta per a un problema formal. Per a resoldre un problema nou crearem una classe heretada, sobrescrivint `accions` i `accio`, i potser altres mètodes.
    Per defecte no hi ha definida cap heurística (h torna sempre 0), i tots els costos d'acció són 1.
"""

    def __init__(self, inicial=None, final=None, **kwds):
        """El constructor especifica l'estat inicial i final (si es coneix) i pot contenir altres arguments."""
        self.__dict__.update(inicial=inicial, final=final, **kwds)

    def accions(self, estat):
        """Retorna les accions possibles que es poden fer en l'estat `estat`."""
        raise NotImplementedError

    def accio(self, estat, action):
        """Retorna l'estat que resulta de fer l'acció `accio` en l'estat `estat`."""
        raise NotImplementedError

    def es_resultat(self, estat):
        """
        Retorna True si l'estat `estat` és un estat objectiu. Per defecte, compara amb l'estat final, si s'ha especificat.
        """
        return estat == self.final

    def cost_accio(self, estat, a, estat1):
        """
        Retorna el cost de fer l'acció `accio` en l'estat `estat` fins a arribar a l'estat `estat1`. Per defecte, retorna 1.
        """
        return 1

    def h(self, estat):
        """
        Retorna un valor heurístic per a l'estat `estat`. Per defecte, retorna 0.
        Per ser admissible, aquest valor heurístic no pot sobreestimar el cost per arribar a l'estat objectiu.
        """
        return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.inicial, self.final)

Aquest codi defineix una classe base anomenada `Problema` per a la resolució de problemes de cerca en Python. La classe `Problema` és una classe abstracta que defineix la interfície per a un problema de cerca genèric. Aquesta classe té diversos mètodes que es poden sobreescriure per a adaptar-se a un problema específic.

Els mètodes de la classe `Problema` són:

- `__init__(self, inicial=None, final=None, **kwds)`: Aquest és el constructor de la classe. Accepta dos paràmetres: `inicial` i `final`, que representen l'estat inicial i final del problema, respectivament. També accepta arguments de paraula clau addicionals que es poden utilitzar per a personalitzar la instància del problema.

- `accions(self, estat)`: Aquest mètode ha de ser sobrescrit per a retornar les accions possibles que es poden realitzar en un estat donat.

- `accio(self, estat, action)`: Aquest mètode ha de ser sobrescrit per a retornar l'estat resultant de realitzar una acció en un estat donat.

- `es_resultat(self, estat)`: Aquest mètode retorna `True` si l'estat donat és un estat final. Per defecte, compara l'estat amb l'estat final especificat en el constructor.

- `cost_accio(self, estat, a, estat1)`: Aquest mètode retorna el cost de realitzar una acció en un estat per a arribar a un altre estat. Per defecte, retorna 1.

- `h(self, estat)`: Aquest mètode retorna un valor heurístic per a un estat. Per defecte, retorna 0. Aquest mètode es pot sobreescriure per a proporcionar una heurística específica per a un problema.

- `__str__(self)`: Aquest mètode retorna una representació en cadena de caràcters de la instància del problema.

Nosaltres anem a solucionar el problema de les torres de Hanoi. Per això, crearem una classe que hereta de la classe `Problema` i sobreescriurem els mètodes que necessitem. 

In [212]:
class ProblemaHanoi(Problema):
    """
    Problema de les torres de Hanoi: donada una configuració inicial, trobar una seqüència d'accions per a arribar a una configuració final.
    """

    def accions(self, estat):
        """
        Retorna les accions possibles en un estat donat. Les accions són parells de nombres que representen les torres d'origen i destí.
        """
        estats = []
        for i in range(3):
            for j in range(3):
                if i != j and self.es_valida(estat, i, j):
                    estats.append((i, j))
        return estats

    def accio(self, estat, accio):
        """
        Retorna l'estat que resulta de fer una acció en un estat donat. 
        """
        i, j = accio
        nou_estat = EstatHanoi([list(torre) for torre in estat.torres], pare=estat)
        nou_estat.torres[j].append(nou_estat.torres[i].pop())
        return nou_estat

    def es_valida(self, estat, i, j):
        """
        Retorna True si es pot moure un disc de la torre i a la torre j.
        Es pot moure un disc de la torre i a la torre j si la torre i no està buida i la torre j està buida o el disc de la torre i és més petit que el disc de la torre j.
        """
        if len(estat.torres[i]) == 0:
            return False
        if len(estat.torres[j]) == 0:
            return True
        return estat.torres[i][-1] < estat.torres[j][-1]

    def es_resultat(self, estat):
        """
        Retorna True si l'estat és un estat final.
        """
        return estat == self.final

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.inicial, self.final)

### Definició dels estats inicial i final i instància del problema

Per a poder resoldre el problema, hem de definir l'estat inicial i l'estat final. En aquest cas, l'estat inicial és una llista on en la primera posició hi ha tots els discos, i en les altres dues posicions hi ha llistes buides. L'estat final és una llista on en l'última posició hi ha tots els discos, i en les altres dues posicions hi ha llistes buides.

A continuació, crearem una instància del problema amb l'estat inicial i final que hem definit.

In [213]:
# Exemple de problema de les torres de Hanoi
estat_inicial = EstatHanoi([[5, 4, 3, 2, 1], [], []])
estat_final = EstatHanoi([[], [], [5, 4, 3, 2, 1]])

p = ProblemaHanoi(estat_inicial, estat_final)
print(p)

ProblemaHanoi([5, 4, 3, 2, 1]
[]
[]
-----
, []
[]
[5, 4, 3, 2, 1]
-----
)


### Implementació de l'algorisme de cerca de cost uniforme

A continuació, implementarem l'algorisme de cerca de cost uniforme. Aquest algorisme consisteix en anar expandint els nodes amb menor cost fins a trobar un node objectiu. Per a implementar aquest algorisme, utilitzarem una cua de prioritat. Aquesta cua ens permetrà afegir els estats a expandir i extreure l'estat amb menor cost. 

Podriem utilitzar l'algorisme de cerca en amplada, però el algorisme de cost uniforme es més general.

In [214]:
import heapq


def cerca_cost_uniforme(problema):
    """
    Cerca de cost uniforme: expandeix l'estat amb menor cost fins a trobar un node objectiu.
    """

    # Inicialitzem la frontera amb l'estat inicial
    inicial = problema.inicial

    # La frontera és una cua de prioritat on el cost del node és la prioritat de la cua. El cost del node és el cost del camí per arribar al node (inicialment 0).
    frontera = [(0, inicial)]

    # Inicialitzem el conjunt de nodes visitats
    visitats = set()

    # Mentre la frontera no estigui buida
    while frontera:
        # Extreiem el node amb menor cost de la frontera
        cost, estat = heapq.heappop(frontera)
        # Afegim l'estat del node als visitats
        visitats.add(estat)
        # Impimim l'estat
        print(estat)

        # Si l'estat és un estat final, retornem el node
        if problema.es_resultat(estat):
            # Imprimim el nombre de nodes visitats
            print(f"Nombre de nodes visitats: {len(visitats)}")

            # Retornem l'estat
            return estat

        # Per a cada acció possible en l'estat
        for accio in problema.accions(estat):
            # Calculem el nou estat
            nou_estat = problema.accio(estat, accio)
            print(nou_estat)
            # Si el nou estat no està visitat
            if nou_estat not in visitats:
                # Afegim el nou estat a la frontera amb el cost del camí acumulat
                cost_accio = problema.cost_accio(estat, accio, nou_estat)
                heapq.heappush(
                    frontera,
                    (cost + cost_accio, nou_estat)
                )

In [215]:
# Exemple de cerca de cost uniforme
desti = cerca_cost_uniforme(p)
desti

[5, 4, 3, 2, 1]
[]
[]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3]
[2]
[1]
-----

[5, 4, 3, 2, 1]
[]
[]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3, 2, 1]
[]
[]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3, 1]
[]
[2]
-----

[5, 4, 3]
[]
[2, 1]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3]
[2]
[1]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3, 1]
[2]
[]
-----

[5, 4, 3]
[2, 1]
[]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3, 2, 1]
[]
[]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3]
[]
[2, 1]
-----

[5, 4]
[3]
[2, 1]
-----

[5, 4, 3, 1]
[]
[2]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3]
[2, 1]
[]
-----

[5, 4]
[2, 1]
[3]
-----

[5, 4, 3, 1]
[2]
[]
-----

[5, 4, 3]
[2]
[1]
-----

[5, 4, 3, 1]
[]
[2]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3]
[]
[2, 1]
-----

[5, 4, 3, 1]
[2]
[]
-----

[5, 4, 3, 1]
[2]
[]
-----

[5, 4, 3]
[2, 1]
[]
-----

[5, 4, 3]
[2]

[]
[]
[5, 4, 3, 2, 1]
-----

### Recuperació del camí

Per a recuperar el camí, anirem des del node objectiu fins al node inicial seguint els punts de l'atribut `pare` de cada node.

A continuació, definirem una funció que ens permeti recuperar el camí.

In [216]:
def recuperar_cami(estat):
    """
    Retorna el camí des de l'estat `estat` fins a l'estat inicial.
    """
    cami = []
    while estat:
        cami.append(estat)
        estat = estat.pare
    return cami[::-1]

In [217]:
# Exemple de recuperació del camí
recuperar_cami(desti)

[[5, 4, 3, 2, 1]
 []
 []
 -----,
 [5, 4, 3, 2]
 []
 [1]
 -----,
 [5, 4, 3]
 [2]
 [1]
 -----,
 [5, 4, 3]
 [2, 1]
 []
 -----,
 [5, 4]
 [2, 1]
 [3]
 -----,
 [5, 4, 1]
 [2]
 [3]
 -----,
 [5, 4, 1]
 []
 [3, 2]
 -----,
 [5, 4]
 []
 [3, 2, 1]
 -----,
 [5]
 [4]
 [3, 2, 1]
 -----,
 [5]
 [4, 1]
 [3, 2]
 -----,
 [5, 2]
 [4, 1]
 [3]
 -----,
 [5, 2, 1]
 [4]
 [3]
 -----,
 [5, 2, 1]
 [4, 3]
 []
 -----,
 [5, 2]
 [4, 3]
 [1]
 -----,
 [5]
 [4, 3, 2]
 [1]
 -----,
 [5]
 [4, 3, 2, 1]
 []
 -----,
 []
 [4, 3, 2, 1]
 [5]
 -----,
 [1]
 [4, 3, 2]
 [5]
 -----,
 [1]
 [4, 3]
 [5, 2]
 -----,
 []
 [4, 3]
 [5, 2, 1]
 -----,
 [3]
 [4]
 [5, 2, 1]
 -----,
 [3]
 [4, 1]
 [5, 2]
 -----,
 [3, 2]
 [4, 1]
 [5]
 -----,
 [3, 2, 1]
 [4]
 [5]
 -----,
 [3, 2, 1]
 []
 [5, 4]
 -----,
 [3, 2]
 []
 [5, 4, 1]
 -----,
 [3]
 [2]
 [5, 4, 1]
 -----,
 [3]
 [2, 1]
 [5, 4]
 -----,
 []
 [2, 1]
 [5, 4, 3]
 -----,
 [1]
 [2]
 [5, 4, 3]
 -----,
 [1]
 []
 [5, 4, 3, 2]
 -----,
 []
 []
 [5, 4, 3, 2, 1]
 -----]

### Utilització d'una heurística

Per no expandir estats que no ens portaran a cap lloc, utilitzarem una funció heurística que ens permeti estimar el cost per arribar a un estat objectiu. Aquesta funció heurística ha de ser admissible, és a dir, no pot sobreestimar el cost per arribar a l'estat objectiu.

En el cas de les torres de Hanoi, una heurística admissible és el nombre de discos que falten per moure. Aquesta heurística és admissible perquè cada acció mou un disc, i el cost de moure un disc és 1. Per tant, el cost per arribar a l'estat objectiu serà sempre igual o superior al nombre de discos que falten per moure.

Implementarem una classe `ProblemaHanoiHeuristic` que hereta de la classe `ProblemaHanoi` i sobreescriurem el mètode `h` per a que retorni el nombre de discos que falten per moure.

In [218]:
class ProblemaHanoiHeuristic(ProblemaHanoi):
    """
    Problema de les torres de Hanoi: donada una configuració inicial, trobar una seqüència d'accions per a arribar a una configuració final.
    """

    def h(self, estat):
        """
        Retorna el nombre de discos que falten per moure.
        """
        return len(estat.torres[0] + estat.torres[1])

### Implementació de l'algorisme A*

Per utilitzar l'heurística, implementarem l'algorisme A*. Aquest algorisme consisteix en expandir els nodes amb menor cost + heurística fins a trobar un node objectiu. 
 

In [219]:
def a_estrella(problema):
    """
    Cerca A*: expandeix l'estat amb menor cost + heurística fins a trobar un node objectiu.
    """

    # Inicialitzem la frontera amb l'estat inicial
    inicial = problema.inicial

    # La frontera és una cua de prioritat on el cost del node és la prioritat de la cua. El cost del node és el cost del camí per arribar al node (inicialment 0).
    frontera = [(0, inicial)]

    # Inicialitzem el conjunt de nodes visitats
    visitats = set()

    # Mentre la frontera no estigui buida
    while frontera:
        # Extreiem el node amb menor cost de la frontera
        cost, estat = heapq.heappop(frontera)
        # Afegim l'estat del node als visitats
        visitats.add(estat)
        # Impimim l'estat
        print(estat)

        # Si l'estat és un estat final, retornem el node
        if problema.es_resultat(estat):
            # Imprimim el nombre de nodes visitats
            print(f"Nombre de nodes visitats: {len(visitats)}")

            # Retornem l'estat
            return estat

        # Per a cada acció possible en l'estat
        for accio in problema.accions(estat):
            # Calculem el nou estat
            nou_estat = problema.accio(estat, accio)
            print(nou_estat)
            # Si el nou estat no està visitat
            if nou_estat not in visitats:
                # Afegim el nou estat a la frontera amb el cost del camí acumulat
                cost_accio = problema.cost_accio(estat, accio, nou_estat)
                heapq.heappush(
                    frontera,
                    (cost + cost_accio + problema.h(nou_estat), nou_estat)
                )

In [220]:
# Exemple de cerca A*
p = ProblemaHanoiHeuristic(estat_inicial, estat_final)
desti = a_estrella(p)

[5, 4, 3, 2, 1]
[]
[]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3]
[2]
[1]
-----

[5, 4, 3, 2, 1]
[]
[]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3, 2, 1]
[]
[]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3]
[2]
[1]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3, 1]
[2]
[]
-----

[5, 4, 3]
[2, 1]
[]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3, 1]
[]
[2]
-----

[5, 4, 3]
[]
[2, 1]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3, 2]
[1]
[]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3, 2, 1]
[]
[]
-----

[5, 4, 3, 2]
[]
[1]
-----

[5, 4, 3]
[]
[2, 1]
-----

[5, 4]
[3]
[2, 1]
-----

[5, 4, 3, 1]
[]
[2]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3]
[2, 1]
[]
-----

[5, 4]
[2, 1]
[3]
-----

[5, 4, 3, 1]
[2]
[]
-----

[5, 4, 3]
[2]
[1]
-----

[5, 4, 3, 1]
[]
[2]
-----

[5, 4, 3]
[1]
[2]
-----

[5, 4, 3]
[]
[2, 1]
-----

[5, 4, 3, 1]
[2]
[]
-----

[5, 4, 3, 1]
[2]
[]
-----

[5, 4, 3]
[2, 1]
[]
-----

[5, 4, 3]
[2]

## Ruta per Castelló

Veurem un altre exemple de cerca amb el problema del viatjer per Castelló. En aquest cas, el problema consisteix en trobar el camí més curt per anar de la UJI a un altre punt de Castelló. Per això, utilitzarem un mapa simplificat de Castelló amb les distàncies entre els diferents punts.

### Definició de l'estat

Per a poder resoldre un problema de cerca, primer hem de definir l'estat del problema. En aquest cas, l'estat del problema és la posició del viatjer en el mapa. Per això, definirem una classe amb aquesta informació.

In [221]:
from dataclasses import dataclass


@dataclass
class EstatCastello(object):
    """
    Estat del problema del viatjer per Castelló: guardarem un enter amb la posició del viatjer i un punter a l'estat pare.
    Definirem també el mètode __str__ per a poder imprimir l'estat, el hash per a poder utilitzar-lo en un set, i els mètodes __eq__ i __lt__ per a poder comparar dos estats.
    """
    posicio: int
    pare: 'EstatCastello' = None

    def __repr__(self):
        """
        Retorna una cadena de caràcters amb la representació de l'estat.
        """
        return str(self.posicio)

    def __hash__(self):
        """
        Retorna un hash (nombre enter cqlculat) que representa l'estat.
        """
        return hash(self.posicio)

    def __eq__(self, altre):
        """
        Retorna True si l'estat és igual a l'estat `altre`. No inclou el pare.
        """
        return self.posicio == altre.posicio

    def __lt__(self, altre):
        """
        Retorna True si l'estat és menor que l'estat `altre`. No inclou el pare.
        """
        return self.posicio < altre.posicio

## Definició del problema

A continuació, crearem una classe que hereta de la classe `Problema` i sobreescriurem els mètodes que necessitem.

In [222]:
class ProblemaCastello(Problema):
    """
    Problema del viatjer per Castelló: donada una posició inicial, trobar una seqüència d'accions per a arribar a una posició final.
    """

    def accions(self, estat):
        """
        Retorna les accions possibles en un estat donat. Les accions són parells de nombres que representen les posicions d'origen i destí.
        Utilitzarem una llista amb les distàncies reals precalculades per a cada punt.
        """
        estats = []
        for i in range(len(self.distancies)):
            if self.distancies[estat.posicio][i] > 0:
                estats.append((estat.posicio, i))
        return estats

    def accio(self, estat, accio):
        """
        Retorna l'estat que resulta de fer una acció en un estat donat. 
        """
        nou_estat = EstatCastello(accio[1], pare=estat)
        return nou_estat

    def es_resultat(self, estat):
        """
        Retorna True si l'estat és un estat final.
        """
        return estat == self.final

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.inicial, self.final)

## Definició dels estats inicial i final i instància del problema

Per a poder resoldre el problema, hem de definir l'estat inicial i l'estat final. En aquest cas, l'estat inicial és la posició de la UJI, i l'estat final és la posició del punt de Castelló on volem anar; en aquest cas, el parc Ribalta.

A continuació, crearem una instància del problema amb l'estat inicial i final que hem definit.

In [223]:
# Exemple de problema del viatjer per Castelló
distancies = [
    [0, 2, 3, 5, 6, 5, 6, 7, 8, 9],  # UJI
    [2, 0, 1, 3, 4, 3, 4, 5, 6, 7],  # Plaça Borrull
    [3, 1, 0, 2, 3, 2, 3, 4, 5, 6],  # Plaça Maria Agustina
    [5, 3, 2, 0, 1, 2, 3, 4, 5, 6],  # Plaça Santa Clara
    [6, 4, 3, 1, 0, 1, 2, 3, 4, 5],  # Plaça Major
    [5, 3, 2, 2, 1, 0, 1, 2, 3, 4],  # Plaça Hort dels Corders
    [6, 4, 3, 3, 2, 1, 0, 1, 2, 3],  # Plaça Huerto Sogueros
    [7, 5, 4, 4, 3, 2, 1, 0, 1, 2],  # Plaça Fadrell
    [8, 6, 5, 5, 4, 3, 2, 1, 0, 1],  # Plaça Tetuán
    [9, 7, 6, 6, 5, 4, 3, 2, 1, 0],  # Parc Ribalta

]
estat_inicial = EstatCastello(0)
estat_final = EstatCastello(9)

p = ProblemaCastello(estat_inicial, estat_final, distancies=distancies)

print(p)

ProblemaCastello(0, 9)


## Implementació de l'algorisme de cerca de cost uniforme

A continuació, aplicarem l'algorisme de cerca de cost uniforme. 

In [224]:
desti = cerca_cost_uniforme(p)
desti

0
1
2
3
4
5
6
7
8
9
1
0
2
3
4
5
6
7
8
9
2
0
1
3
4
5
6
7
8
9
3
0
1
2
4
5
6
7
8
9
4
0
1
2
3
5
6
7
8
9
5
0
1
2
3
4
6
7
8
9
6
0
1
2
3
4
5
7
8
9
7
0
1
2
3
4
5
6
8
9
8
0
1
2
3
4
5
6
7
9
9
Nombre de nodes visitats: 10


9

In [225]:
recuperar_cami(desti)

[0, 9]

## Utilització d'una heurística

En el cas del viatjer per Castelló, una heurística admissible és la distància Manhattan fins al punt final. Aquesta heurística és admissible perquè cada acció mou el viatjer a un punt més proper al punt final, i el cost de moure el viatjer és 1. Per tant, el cost per arribar a l'estat objectiu serà sempre igual o superior a la distància Manhattan fins al punt final.

Implementarem una classe `ProblemaCastelloHeuristic` que hereta de la classe `ProblemaCastello` i sobreescriurem el mètode `h` per a que retorni la distància en línia recta fins al punt final.


In [226]:
class ProblemaCastelloHeuristic(ProblemaCastello):
    """
    Problema del viatjer per Castelló: donada una posició inicial, trobar una seqüència d'accions per a arribar a una posició final.
    """

    def h(self, estat):
        """
        Retorna la distància Manhattan fins al punt final.
        """
        return abs(estat.posicio - self.final.posicio)

In [227]:
# Exemple de cerca A*
p = ProblemaCastelloHeuristic(estat_inicial, estat_final, distancies=distancies)
desti = a_estrella(p)
recuperar_cami(desti)

0
1
2
3
4
5
6
7
8
9
9
Nombre de nodes visitats: 2


[0, 9]